In [10]:
import torch
import data_reading
import os
import cv2
import numpy as np
from scipy import signal
import pickle

In [15]:
image_width = 128
image_height = 96
labels_folder = '../data/labels'
imgs_folder = '../data/images'
video_folder = '../data/training_videos'

In [28]:
def read_video(file_name):
    cap = cv2.VideoCapture(file_name)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    buf = np.empty((frameCount, 3, frameHeight, frameWidth), np.dtype('uint8'))
    fc = 0
    ret = True
    SAMPLING_FREQUENCY = cap.get(cv2.CAP_PROP_FPS)
    print(frameCount/SAMPLING_FREQUENCY)
    while (fc < frameCount  and ret):
        ret, img = cap.read()
        buf[fc] = np.transpose(img, (2,0,1))
        fc += 1
    cap.release()
    freqs, times, spectr = signal.spectrogram(buf, fs=SAMPLING_FREQUENCY, window=('hamming'), noverlap=13, nperseg=14, axis=0, mode='magnitude')
    return freqs, times, spectr

In [31]:
for video_path in os.listdir(video_folder):
    if 'W' in video_path or 'NW' in video_path:
        freqs, times, spectr = read_video(f'{video_folder}/{video_path}')
with open('data/freqs.data','wb') as f:
    pickle.dump(freqs, f)
with open('data/times.data','wb') as f:
    pickle.dump(times, f)
with open('data/spectr', 'wb') as f:
    pickle.dump(spectr, f)

19.066666666666666
29.866666666666667


In [ ]:
for folder in os.listdir('data/labels'):
    if folder != '.DS_Store':
        for filename in os.listdir(f"{labels_folder}{folder}"):
            if filename.endswith("YOLO"):
                for _file in os.listdir(f"{labels_folder}{folder}/{filename}"):
                    with open(f"{labels_folder}{folder}/{filename}/{_file}") as f:
                        boxes = []
                        labels = []
                        for line in f:
                            line = line.split()
                            label, rest = int(line[0]), line[1:]
                            xmin, ymin, w, h = map(float, rest)
                            xmin, ymin = int(xmin*image_width), int(ymin*image_height)
                            w, h = int(w*image_width), int(h*image_height)
                            boxes.append(torch.tensor([xmin, ymin, min(w+xmin, image_width-1), min(h+ymin, image_height-1)]))
                            labels.append(label)                                     
                        
                        self.data[-1][1] = {'boxes':torch.stack(boxes).float(), 'labels':torch.tensor(labels).long()+1}
                    image = torch.tensor(cv2.cvtColor(cv2.imread(f"{images_folder}/{folder}/{_file[:-3]}jpg", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB))
                    image = image.permute(2,0,1).double()
                    self.data[-1][0] = image